# ✨ Python Decorators: Enhancing Code with Syntactic Sugar

**Welcome!** Decorators are one of Python's most powerful and elegant features, allowing you to modify or enhance functions and methods in a clean, readable way. They are a form of metaprogramming, leveraging Python's support for functions as first-class objects. This notebook provides a detailed exploration of how decorators work, how to create them, and their common use cases.

**Target Audience:** Python developers wanting to understand and utilize decorators effectively to write more reusable, maintainable, and expressive code.

**Learning Objectives:**
*   Understand the concept of functions as first-class citizens in Python.
*   Learn the basic syntax and mechanics of function decorators (`@` syntax).
*   Implement simple decorators and understand the role of wrapper functions.
*   Properly handle function arguments (`*args`, `**kwargs`) and return values in decorators.
*   Preserve original function metadata using `@functools.wraps`.
*   Create decorators that accept arguments using decorator factories.
*   Implement decorators using classes.
*   Apply decorators to classes.
*   Explore practical use cases like logging, timing, access control, and caching.
*   Identify best practices and common pitfalls.

## 1. Foundation: Functions as First-Class Objects

Before diving into decorators, it's crucial to understand that functions in Python are treated like any other object (integers, strings, lists, etc.). This means they can be:

1.  **Assigned to variables.**
2.  **Passed as arguments** to other functions.
3.  **Returned from** other functions.

This capability is what makes decorators possible.

In [1]:
def shout(text: str) -> str:
    """Converts text to uppercase with an exclamation mark."""
    return text.upper() + "!"

# 1. Assign function to a variable
yell = shout
print(f"Calling via 'yell': {yell('hello')}")

# 2. Pass function as an argument
def apply_func(func, value):
    """Applies a function to a value."""
    return func(value)

print(f"Applying shout via apply_func: {apply_func(shout, 'greetings')}")

# 3. Return a function from another function
def create_adder(x):
    """Returns a function that adds 'x' to its argument."""
    def adder(y):
        return x + y
    return adder

add_5 = create_adder(5)
add_10 = create_adder(10)

print(f"Calling add_5(10): {add_5(10)}")
print(f"Calling add_10(10): {add_10(10)}")

Calling via 'yell': HELLO!
Applying shout via apply_func: GREETINGS!
Calling add_5(10): 15
Calling add_10(10): 20


## 2. What is a Decorator?

A decorator is essentially a function that takes another function (the decorated function) as input, adds some functionality to it without modifying its source code, and returns the enhanced function.

**Analogy: Function Wrapping Paper (Revisited)**
As mentioned before, think of a decorator (`@my_decorator`) as special wrapping applied to a function (`def my_func(): ...`). The original function (`my_func`) is the gift inside. The decorator adds behavior (logging, timing, access checks) like adding layers of wrapping paper, ribbons, or labels before the gift is finally presented (the function is called).

**The `@` Syntax:**
The `@decorator_name` syntax placed directly above a function definition is Python's syntactic sugar for this process.

### 2.1 A Simple Decorator: Manual Application

Let's first see how it works *without* the `@` syntax.

In [2]:
import logging
from typing import Callable, Any

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s', force=True)

# The Decorator Function
def simple_logger_decorator(original_func: Callable) -> Callable:
    """A simple decorator that logs before and after function execution."""
    print(f"---> Decorator simple_logger_decorator is initializing for {original_func.__name__}")
    
    # The Inner Wrapper Function
    def wrapper_function(*args: Any, **kwargs: Any) -> Any:
        """This function wraps the original function's execution."""
        logging.info(f"Executing '{original_func.__name__}'...")
        
        # Call the original function passed to the decorator
        result = original_func(*args, **kwargs)
        
        logging.info(f"Finished executing '{original_func.__name__}'. Result: {result!r}")
        return result
        
    # Return the wrapper function
    return wrapper_function

# The Original Function
def add(x: int, y: int) -> int:
    """Adds two numbers."""
    return x + y

# --- Manually Applying the Decorator --- 
print("--- Manual Decoration --- ")
print(f"Original add function: {add}")

# Pass the original function to the decorator
# The decorator returns the 'wrapper_function'
decorated_add = simple_logger_decorator(add) 

print(f"Decorated add function: {decorated_add}") # Now points to the wrapper

# Call the decorated version
sum_result = decorated_add(10, 20)
print(f"Final Sum: {sum_result}")

# Calling the original 'add' directly would NOT have the logging
# original_sum = add(5, 3)
# print(f"Original Sum (no logging): {original_sum}")

INFO: Executing 'add'...
INFO: Finished executing 'add'. Result: 30


--- Manual Decoration --- 
Original add function: <function add at 0x78f4ca993560>
---> Decorator simple_logger_decorator is initializing for add
Decorated add function: <function simple_logger_decorator.<locals>.wrapper_function at 0x78f4ca9920c0>
Final Sum: 30


### 2.2 Using the `@` Syntax

The `@` syntax simplifies the manual application process.

In [3]:
@simple_logger_decorator # Apply decorator using '@'
def subtract(x: int, y: int) -> int:
    """Subtracts two numbers."""
    return x - y

# Now, calling subtract() automatically calls the decorated version
print("\n--- Decorating with @ syntax ---")
diff_result = subtract(50, 15)
print(f"Final Difference: {diff_result}")

INFO: Executing 'subtract'...
INFO: Finished executing 'subtract'. Result: 35


---> Decorator simple_logger_decorator is initializing for subtract

--- Decorating with @ syntax ---
Final Difference: 35


## 3. Handling Arguments and Return Values

Decorators need to handle arbitrary arguments (`*args`, `**kwargs`) and return values from the decorated function correctly.

Our `simple_logger_decorator` already does this:
*   `def wrapper_function(*args: Any, **kwargs: Any)`: Accepts any positional and keyword arguments.
*   `result = original_func(*args, **kwargs)`: Passes these arguments down to the original function.
*   `return result`: Returns whatever the original function returned.

## 4. Preserving Function Metadata: `@functools.wraps`

**Problem:** When you decorate a function, the returned object is the *wrapper* function. This means introspection tools (like `help()`, debuggers) see the wrapper's name and docstring, not the original function's.

**Solution:** Use the `@functools.wraps` decorator *inside* your decorator, applying it to the wrapper function. It copies metadata (`__name__`, `__doc__`, `__annotations__`, etc.) from the original function to the wrapper.

In [4]:
import functools
import logging

def proper_logger_decorator(func: Callable) -> Callable:
    """A decorator that logs AND preserves metadata."""
    
    @functools.wraps(func) # Apply wraps to the wrapper
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        """This is the wrapper's docstring (now hidden)."""
        logger = logging.getLogger(func.__qualname__) # Use original func name for logger
        logger.info(f"Executing '{func.__name__}'...")
        result = func(*args, **kwargs)
        logger.info(f"Finished '{func.__name__}'. Result: {result!r}")
        return result
    return wrapper

@proper_logger_decorator
def multiply(x: int, y: int) -> int:
    """Multiplies two numbers. This docstring should be preserved."""
    return x * y

print("--- Testing @wraps --- ")
prod_result = multiply(6, 7)
print(f"Final Product: {prod_result}")

# Check metadata
print(f"\nFunction name: {multiply.__name__}") # Shows 'multiply', not 'wrapper'
print(f"Function docstring: {multiply.__doc__}") # Shows original docstring
print(f"Function annotations: {multiply.__annotations__}") # Preserves type hints

# Compare with the earlier 'subtract' function (which didn't use wraps)
# help(subtract) # Would show info about 'wrapper_function'
help(multiply) # Shows info about 'multiply'

INFO: Executing 'multiply'...
INFO: Finished 'multiply'. Result: 42


--- Testing @wraps --- 
Final Product: 42

Function name: multiply
Function docstring: Multiplies two numbers. This docstring should be preserved.
Function annotations: {'x': <class 'int'>, 'y': <class 'int'>, 'return': <class 'int'>}
Help on function multiply in module __main__:

multiply(x: int, y: int) -> int
    Multiplies two numbers. This docstring should be preserved.



**Best Practice:** *Always* use `@functools.wraps` when writing decorators.

## 5. Decorators with Arguments (Decorator Factories)

Sometimes you want to configure the decorator itself (e.g., specify a log level, a number of retries). This requires a **decorator factory**: a function that takes the arguments and *returns* the actual decorator.

In [5]:
import functools
import time
from typing import Callable, Any

DEFAULT_RETRIES = 3
DEFAULT_DELAY = 1.0

# Decorator Factory Function
def retry(num_retries: int = DEFAULT_RETRIES, delay: float = DEFAULT_DELAY, 
          allowed_exceptions: tuple = (Exception,)) -> Callable:
    """Returns a decorator that retries a function on specified exceptions."""
    
    print(f"---> Initializing retry decorator with retries={num_retries}, delay={delay}")
    
    # The Actual Decorator
    def decorator_retry(func: Callable) -> Callable:
        @functools.wraps(func)
        def wrapper_retry(*args: Any, **kwargs: Any) -> Any:
            logger = logging.getLogger(func.__qualname__)
            attempts = 0
            while attempts < num_retries:
                attempts += 1
                try:
                    logger.info(f"Attempt {attempts}/{num_retries}: Calling {func.__name__}")
                    result = func(*args, **kwargs)
                    logger.info(f"Attempt {attempts} succeeded.")
                    return result # Success, exit loop
                except allowed_exceptions as e:
                    logger.warning(f"Attempt {attempts} failed with {type(e).__name__}: {e}")
                    if attempts == num_retries:
                        logger.error(f"Function {func.__name__} failed after {num_retries} attempts.")
                        raise # Re-raise the last exception
                    else:
                        logger.info(f"Retrying in {delay:.2f} seconds...")
                        time.sleep(delay)
            # This part should ideally not be reached if retries >= 1
            # It's a fallback, but the raise in the except block is primary
            logger.critical(f"Exited retry loop unexpectedly for {func.__name__}")
            # Depending on desired behavior, could return None or raise here too
            # raise RuntimeError("Function failed after all retries.")
        return wrapper_retry
    return decorator_retry

# --- Using the Decorator Factory --- 
fail_counter = 0

# Use default arguments (3 retries, 1s delay, catches Exception)
# @retry()
# def potentially_flaky_operation_default():
#     pass

# Specify arguments
@retry(num_retries=4, delay=0.1, allowed_exceptions=(ConnectionError, TimeoutError))
def connect_to_service(url: str):
    """Simulates connecting to a service that might fail."""
    global fail_counter
    print(f"Attempting connection to {url}... (Fail counter: {fail_counter})" )
    if fail_counter < 3:
        fail_counter += 1
        raise ConnectionError(f"Simulated connection failure #{fail_counter}")
    else:
        print(f"Connection to {url} established successfully.")
        return {"status": "connected", "url": url}

print("\n--- Testing @retry decorator --- ")
try:
    connection_status = connect_to_service("http://example-service.com")
    print(f"Final connection status: {connection_status}")
except Exception as e:
    print(f"Caught final exception after retries: {type(e).__name__}: {e}")

# Example that would fail permanently
fail_counter = 0
print("\n--- Testing @retry with permanent failure ---")
try:
    # This will use defaults (3 retries) but always raise ConnectionError
    @retry()
    def always_fail():
        global fail_counter
        fail_counter += 1
        raise ConnectionError(f"Permanent failure #{fail_counter}")
    always_fail()
except ConnectionError as e:
    print(f"Caught expected permanent failure: {e}")

INFO: Attempt 1/4: Calling connect_to_service
INFO: Retrying in 0.10 seconds...


---> Initializing retry decorator with retries=4, delay=0.1

--- Testing @retry decorator --- 
Attempting connection to http://example-service.com... (Fail counter: 0)


INFO: Attempt 2/4: Calling connect_to_service


Attempting connection to http://example-service.com... (Fail counter: 1)


INFO: Retrying in 0.10 seconds...
INFO: Attempt 3/4: Calling connect_to_service
INFO: Retrying in 0.10 seconds...


Attempting connection to http://example-service.com... (Fail counter: 2)


INFO: Attempt 4/4: Calling connect_to_service
INFO: Attempt 4 succeeded.
INFO: Attempt 1/3: Calling always_fail
INFO: Retrying in 1.00 seconds...


Attempting connection to http://example-service.com... (Fail counter: 3)
Connection to http://example-service.com established successfully.
Final connection status: {'status': 'connected', 'url': 'http://example-service.com'}

--- Testing @retry with permanent failure ---
---> Initializing retry decorator with retries=3, delay=1.0


INFO: Attempt 2/3: Calling always_fail
INFO: Retrying in 1.00 seconds...
INFO: Attempt 3/3: Calling always_fail
ERROR: Function always_fail failed after 3 attempts.


Caught expected permanent failure: Permanent failure #3


## 6. Class-Based Decorators

Decorators can also be implemented using classes, primarily by implementing the `__call__` method, making instances of the class callable.
This is often useful when the decorator needs to maintain state between calls to the decorated function.

In [6]:
import functools
from typing import Callable, Any

class StatefulCounterDecorator:
    def __init__(self, func: Callable):
        functools.update_wrapper(self, func) # Copy metadata
        self.func = func
        self.call_count = 0
        print(f"---> Initializing StatefulCounterDecorator for '{func.__name__}'")

    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        self.call_count += 1
        print(f"---> Call number {self.call_count} to '{self.func.__name__}'")
        result = self.func(*args, **kwargs)
        print(f"<--- Finished call {self.call_count} of '{self.func.__name__}'")
        return result

@StatefulCounterDecorator
def process_item(item_id: int):
    """Processes a single item."""
    print(f"  Processing item {item_id}... done.")
    return f"processed_{item_id}"

print("\n--- Testing Stateful Class Decorator ---")
result_a = process_item(101)
result_b = process_item(102)
print(f"Result A: {result_a}")
print(f"Result B: {result_b}")

# Access the state (call count) via the decorated function object
print(f"Total calls to process_item: {process_item.call_count}")

---> Initializing StatefulCounterDecorator for 'process_item'

--- Testing Stateful Class Decorator ---
---> Call number 1 to 'process_item'
  Processing item 101... done.
<--- Finished call 1 of 'process_item'
---> Call number 2 to 'process_item'
  Processing item 102... done.
<--- Finished call 2 of 'process_item'
Result A: processed_101
Result B: processed_102
Total calls to process_item: 2


## 7. Decorating Classes

Decorators can be applied to class definitions as well. The decorator function receives the class object and should return a class object (usually the same class, potentially modified).
Common uses include registering classes, adding methods/attributes, or wrapping all public methods.

In [7]:
import logging
from typing import Type, Callable, Any
import functools
import time

# Simple decorator to register classes
PLUGIN_REGISTRY = {}

def register_plugin(plugin_id: str) -> Callable[[Type], Type]:
    """Decorator factory to register a class in a registry."""
    def decorator(cls: Type) -> Type:
        logger = logging.getLogger('PluginRegistry')
        logger.info(f"Registering plugin '{plugin_id}' -> {cls.__name__}")
        if plugin_id in PLUGIN_REGISTRY:
            logger.warning(f"Plugin ID '{plugin_id}' already exists. Overwriting.")
        PLUGIN_REGISTRY[plugin_id] = cls
        # Return the original class, unmodified in this case
        return cls 
    return decorator

# Decorator to add logging to all public methods of a class
def log_all_methods(cls: Type) -> Type:
    """Class decorator to wrap public methods with logging."""
    logger = logging.getLogger(f'{cls.__module__}.{cls.__name__}')
    logger.info(f"Applying method logging to class '{cls.__name__}'")
    
    # Simple logging wrapper for methods
    def method_logger_wrapper(method: Callable) -> Callable:
        @functools.wraps(method)
        def wrapper(self, *args, **kwargs):
            logger.debug(f"Calling method {cls.__name__}.{method.__name__}")
            try:
                result = method(self, *args, **kwargs)
                logger.debug(f"Method {cls.__name__}.{method.__name__} finished")
                return result
            except Exception as e:
                logger.exception(f"Exception in method {cls.__name__}.{method.__name__}")
                raise
        return wrapper

    # Iterate over class attributes
    for attr_name, attr_value in vars(cls).items():
        # Check if it's a user-defined function/method (not built-in or dunder)
        if callable(attr_value) and not attr_name.startswith("__"):
            logger.debug(f"  Wrapping method: {attr_name}")
            setattr(cls, attr_name, method_logger_wrapper(attr_value))
            
    return cls

# --- Using Class Decorators --- 
print("--- Defining Classes with Decorators ---")

@register_plugin("data_processor")
@log_all_methods # Apply method logging *after* registration
class DataProcessorPlugin:
    """A plugin to process data."""
    def __init__(self, source: str):
        self.source = source
        self._internal_state = "Initialized"
        # __init__ is usually not wrapped by simple method decorators 
        # as vars(cls) doesn't typically list it directly in this context.
        
    def load_data(self):
        print(f"  Loading data from {self.source}...")
        time.sleep(0.1)
        self._internal_state = "Loaded"
        print(f"  Data loaded.")
        return True

    def process(self):
        if self._internal_state != "Loaded":
            raise RuntimeError("Data must be loaded before processing.")
        print(f"  Processing data...")
        time.sleep(0.2)
        print(f"  Processing complete.")
        return "Processed Results"
    
    # Private/dunder methods are typically ignored by simple decorators
    def _internal_helper(self):
        pass 

@register_plugin("reporter")
class ReportingPlugin:
     """A plugin for reporting."""
     def generate_report(self, data):
         print(f"Generating report for: {data}")
         return "Report Contents"

print("\n--- Using Decorated Classes --- ")
print(f"Plugin Registry: {PLUGIN_REGISTRY}")

# Instantiate and use the plugin
ProcessorClass = PLUGIN_REGISTRY.get("data_processor")
if ProcessorClass:
    # Set logger level to DEBUG to see method logs
    logging.getLogger(f'{ProcessorClass.__module__}.{ProcessorClass.__name__}').setLevel(logging.DEBUG)
    
    processor = ProcessorClass(source="/data/input.csv")
    processor.load_data()
    results = processor.process()
    print(f"Processor results: {results}")
else:
    print("Data processor plugin not found.")

INFO: Applying method logging to class 'DataProcessorPlugin'
INFO: Registering plugin 'data_processor' -> DataProcessorPlugin
INFO: Registering plugin 'reporter' -> ReportingPlugin
DEBUG: Calling method DataProcessorPlugin.load_data
DEBUG: Method DataProcessorPlugin.load_data finished
DEBUG: Calling method DataProcessorPlugin.process


--- Defining Classes with Decorators ---

--- Using Decorated Classes --- 
Plugin Registry: {'data_processor': <class '__main__.DataProcessorPlugin'>, 'reporter': <class '__main__.ReportingPlugin'>}
  Loading data from /data/input.csv...
  Data loaded.
  Processing data...


DEBUG: Method DataProcessorPlugin.process finished


  Processing complete.
Processor results: Processed Results


## 8. Built-in Decorators

Python has several useful built-in decorators:
*   `@property`: Creates read-only properties (can add `@*.setter` and `@*.deleter`). Turns a method into a getter attribute access.
*   `@classmethod`: Defines a method that receives the class (`cls`) as the first argument instead of the instance (`self`).
*   `@staticmethod`: Defines a method that doesn't receive the instance (`self`) or class (`cls`) implicitly. It's like a regular function namespaced within the class.
*   `@dataclasses.dataclass`: (From `dataclasses` module) Automatically adds methods like `__init__`, `__repr__`, `__eq__` to classes.

*(These are often covered in OOP sections, but they are implemented using the decorator mechanism.)*

## 9. Best Practices & Enterprise Considerations

1.  **`@functools.wraps` is Non-Negotiable:** Always use it for decorators modifying functions/methods to preserve introspection.
2.  **Keep Decorators Simple and Focused:** A decorator should ideally have a single, clear responsibility.
3.  **Composability:** Design decorators so they can be stacked effectively (order matters!).
4.  **Use Decorator Arguments for Configurability:** Use the factory pattern when a decorator needs parameters.
5.  **Testing:** Write unit tests specifically for your decorators to ensure they add the correct behavior and handle edge cases. Test decorated functions to ensure their original logic remains intact.
6.  **Readability:** While powerful, complex or deeply nested decorators can sometimes obscure the underlying function's purpose. Comment complex decorators well.
7.  **Framework Usage:** Be aware that many frameworks (web frameworks like Flask/Django, testing frameworks like `pytest`) rely heavily on decorators for configuration, routing, fixtures, etc. Understanding decorators is key to using these frameworks effectively.
8.  **Performance:** Decorators add a function call overhead. This is usually insignificant but measure in performance-critical code if necessary.

## 10. Pitfalls and Common Interview Questions

**Common Pitfalls:**

*   **Forgetting `@functools.wraps`:** Leading to loss of function metadata.
*   **Incorrect Argument Handling:** Wrapper not accepting `*args, **kwargs` or not passing them correctly to the original function.
*   **Not Returning Values:** Wrapper function doesn't return the result of the original function.
*   **Decorator Argument Errors:** Confusing the structure needed for decorators with arguments (requires the extra factory function layer).
*   **Stacking Order:** Applying decorators in the wrong order leading to unexpected behavior.
*   **Stateful Decorators:** Class decorators holding state can sometimes lead to unexpected behavior if not designed carefully, especially if the decorated function is called concurrently.
*   **Debugging Complexity:** Stepping through multiple decorator layers can be confusing.

**Common Interview Questions:**

1.  What is a decorator in Python?
2.  Explain the `@` syntax for decorators.
3.  Why are functions considered first-class objects in Python, and how does this enable decorators?
4.  How do you handle arbitrary arguments (`*args`, `**kwargs`) in a decorator's wrapper function?
5.  What problem does `functools.wraps` solve?
6.  How do you create a decorator that accepts arguments?
7.  Can you implement a decorator using a class? How?
8.  Give some common use cases for decorators.
9.  What happens if you apply multiple decorators to a single function? In what order are they applied?
10. Name some built-in Python decorators (`@property`, `@classmethod`, etc.).

## 11. Challenge: Access Control Decorator

**Goal:** Create a decorator that checks if a user has the required permission level before executing a function.

**Tasks:**

1.  **Define User Roles (Simple):** Use strings like `'GUEST'`, `'USER'`, `'ADMIN'`.
2.  **Simulate Current User:** Create a simple global variable or function `get_current_user_role()` that returns the role of the 'cuurent' user (for testing, you can just return a fixed value or cycle through roles).
3.  **Create Decorator Factory (`require_role`):**
    *   This factory should accept a `required_role` string (e.g., `'ADMIN'`) as an argument.
    *   It should return the actual decorator.
4.  **Implement the Decorator:**
    *   The decorator should take the function to protect as input.
    *   The wrapper function inside the decorator should:
        *   Call `get_current_user_role()` to get the current user's role.
        *   Compare the user's role with the `required_role` passed to the factory. Implement a simple hierarchy (e.g., ADMIN can access USER and GUEST levels, USER can access GUEST).
        *   If the user has sufficient privileges, execute the original function and return its result.
        *   If the user does *not* have sufficient privileges, raise a custom `PermissionError` exception (or print an error message and return `None`).
    *   Ensure the decorator uses `@functools.wraps`.
5.  **Apply and Test:**
    *   Create dummy functions representing actions requiring different permission levels (e.g., `view_public_page`, `edit_user_profile`, `delete_system_data`).
    *   Apply the `@require_role(...)` decorator with appropriate roles to these functions.
    *   Call the decorated functions while simulating different current user roles (by changing what `get_current_user_role()` returns) and verify that access is correctly granted or denied (catching the `PermissionError` or checking the return value).

In [8]:
# --- Solution Space for Challenge ---
import functools
from typing import Callable, Any, Literal

# 1 & 2: Define Roles and Simulate Current User
UserRole = Literal['GUEST', 'USER', 'ADMIN']
ROLE_HIERARCHY = {
    'GUEST': 0,
    'USER': 1,
    'ADMIN': 2
}

_current_user_role: UserRole = 'GUEST' # Global variable for simulation

def get_current_user_role() -> UserRole:
    """Simulates getting the role of the currently logged-in user."""
    # print(f"(Checking role: current is {_current_user_role})")
    return _current_user_role

def set_simulated_role(role: UserRole):
    """Helper to change the simulated role for testing."""
    global _current_user_role
    if role not in ROLE_HIERARCHY:
        raise ValueError(f"Invalid role: {role}")
    _current_user_role = role
    print(f"\n--- Simulated user role set to: {_current_user_role} ---")

# Custom Exception
class PermissionDeniedError(Exception):
    "Raised when a user lacks permission for an action."
    pass

# 3 & 4: Decorator Factory and Decorator Implementation
def require_role(required_role: UserRole) -> Callable:
    """Decorator factory to enforce role-based access control."""
    if required_role not in ROLE_HIERARCHY:
        raise ValueError(f"Invalid required_role specified: {required_role}")
        
    required_level = ROLE_HIERARCHY[required_role]
    
    def decorator(func: Callable) -> Callable:
        @functools.wraps(func)
        def wrapper(*args: Any, **kwargs: Any) -> Any:
            current_role = get_current_user_role()
            current_level = ROLE_HIERARCHY[current_role]
            
            print(f"Checking access for '{func.__name__}': Required='{required_role}'({required_level}), User='{current_role}'({current_level})")
            
            if current_level >= required_level:
                # User has sufficient or higher privileges
                print(f"Access GRANTED for {current_role} to '{func.__name__}'.")
                return func(*args, **kwargs)
            else:
                # User does not have sufficient privileges
                error_msg = f"User role '{current_role}' denied access. Requires '{required_role}'."
                print(f"Access DENIED for {current_role} to '{func.__name__}'.")
                raise PermissionDeniedError(error_msg)
                # Alternatively: return None or some other indicator
        return wrapper
    return decorator

# 5. Apply and Test
@require_role('GUEST') # Everyone can access
def view_public_page():
    print("  -> Viewing public information page.")
    return "Public Content"

@require_role('USER') # Requires USER or ADMIN
def edit_user_profile(username: str):
    print(f"  -> Attempting to edit profile for {username}.")
    return f"Profile for {username} updated."

@require_role('ADMIN') # Requires ADMIN only
def delete_system_data(table: str):
    print(f"  -> Attempting to delete data from table '{table}'.")
    return f"Data from {table} deleted."

print("--- Testing Access Control Decorator ---")

# Test as GUEST
set_simulated_role('GUEST')
view_public_page()
try:
    edit_user_profile("guest123")
except PermissionDeniedError as e:
    print(f"  Caught expected error: {e}")
try:
    delete_system_data("users")
except PermissionDeniedError as e:
    print(f"  Caught expected error: {e}")

# Test as USER
set_simulated_role('USER')
view_public_page()
edit_user_profile("user456")
try:
    delete_system_data("products")
except PermissionDeniedError as e:
    print(f"  Caught expected error: {e}")

# Test as ADMIN
set_simulated_role('ADMIN')
view_public_page()
edit_user_profile("admin007")
delete_system_data("logs")

--- Testing Access Control Decorator ---

--- Simulated user role set to: GUEST ---
Checking access for 'view_public_page': Required='GUEST'(0), User='GUEST'(0)
Access GRANTED for GUEST to 'view_public_page'.
  -> Viewing public information page.
Checking access for 'edit_user_profile': Required='USER'(1), User='GUEST'(0)
Access DENIED for GUEST to 'edit_user_profile'.
  Caught expected error: User role 'GUEST' denied access. Requires 'USER'.
Checking access for 'delete_system_data': Required='ADMIN'(2), User='GUEST'(0)
Access DENIED for GUEST to 'delete_system_data'.
  Caught expected error: User role 'GUEST' denied access. Requires 'ADMIN'.

--- Simulated user role set to: USER ---
Checking access for 'view_public_page': Required='GUEST'(0), User='USER'(1)
Access GRANTED for USER to 'view_public_page'.
  -> Viewing public information page.
Checking access for 'edit_user_profile': Required='USER'(1), User='USER'(1)
Access GRANTED for USER to 'edit_user_profile'.
  -> Attempting to edi

'Data from logs deleted.'

## 12. Conclusion

Decorators are a versatile and expressive feature in Python, enabling powerful patterns like aspect-oriented programming (separating cross-cutting concerns like logging or timing) and functional composition. By understanding how they leverage first-class functions and by using tools like `@functools.wraps`, you can create decorators that enhance your code's functionality while maintaining readability and introspection capabilities.

While initially seeming like magic, decorators are simply a concise syntax for function wrapping. Mastering them allows you to write more DRY, reusable, and declarative Python code, and is essential for working effectively with many modern Python frameworks.